<a href="https://colab.research.google.com/github/Riyasaha256/CSE475_lab1-mango_leaf-/blob/main/Mango_leaf_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sahariya116_mango_leafbd_path = kagglehub.dataset_download('sahariya116/mango-leafbd')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!ls /kaggle/input/


# To check the subdirectories inside MangoLeafBD Dataset

In [ ]:
!ls /kaggle/input/mango-leafbd



In [ ]:
import os

# Set the dataset directory path (change if needed)
data_dir = "/kaggle/input/mango-leafbd"

# Check if the directory exists
if os.path.exists(data_dir):
    print("Dataset is available!")
    print("Contents:", os.listdir(data_dir))  # List files & folders in dataset
else:
    print("Dataset not found. Check the path.")


In [ ]:
import os

data_dir = "/kaggle/input/mango-leafbd"
classes = os.listdir(data_dir)
for cls in classes:
    print(f"Class: {cls}, Number of Images: {len(os.listdir(os.path.join(data_dir, cls)))}")


# List Subdirectories Inside MangoLeafBD Dataset

In [ ]:
import os

data_dir = "/kaggle/input/mango-leafbd/MangoLeafBD Dataset"
classes = os.listdir(data_dir)
for cls in classes:
    print(f"Class: {cls}, Number of Images: {len(os.listdir(os.path.join(data_dir, cls)))}")

# image size

In [ ]:
from PIL import Image

image_sizes = []
for cls in classes:
    for img_file in os.listdir(os.path.join(data_dir, cls)):
        img = Image.open(os.path.join(data_dir, cls, img_file))
        image_sizes.append(img.size)

print(set(image_sizes))  # Unique image sizes

# data distribution

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

class_counts = [len(os.listdir(os.path.join(data_dir, cls))) for cls in classes]
sns.barplot(x=classes, y=class_counts)
plt.xticks(rotation=45)
plt.title("Class Distribution")
plt.show()

In [ ]:
import random
import matplotlib.pyplot as plt

fig, axes = plt.subplots(len(classes), 5, figsize=(15, len(classes) * 3))
for i, cls in enumerate(classes):
    class_dir = os.path.join(data_dir, cls)
    for j in range(5):
        img_path = os.path.join(class_dir, random.choice(os.listdir(class_dir)))
        img = Image.open(img_path)
        axes[i, j].imshow(img)
        axes[i, j].axis("off")
        if j == 0:
            axes[i, j].set_title(cls)
plt.show()

# Data Augmentation

In [ ]:
import os
import torch
from torchvision import transforms, datasets
from PIL import Image
import matplotlib.pyplot as plt
import random

# Path to the Mango Leaf dataset
data_dir = "/kaggle/input/mango-leafbd/MangoLeafBD Dataset"  # Update this path according to your dataset location

# Visualizing a few random images from the Mango Leaf dataset
def visualize_images(data_dir):
    class_names = os.listdir(data_dir)
    fig, axes = plt.subplots(1, len(class_names), figsize=(15, 5))

    for i, class_name in enumerate(class_names):
        class_dir = os.path.join(data_dir, class_name)
        image_file = random.choice(os.listdir(class_dir))
        image_path = os.path.join(class_dir, image_file)

        img = Image.open(image_path)
        axes[i].imshow(img)
        axes[i].set_title(class_name)
        axes[i].axis("off")

    plt.show()

visualize_images(data_dir)

# Data augmentation pipeline
transform_pipeline = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize all images to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomRotation(20),  # Randomly rotate the image by 20 degrees
    transforms.ToTensor(),  # Convert the image to a tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Create a dataset using ImageFolder and apply the transform pipeline
image_dataset = datasets.ImageFolder(root=data_dir, transform=transform_pipeline)
dataloader = torch.utils.data.DataLoader(image_dataset, batch_size=16, shuffle=True)

# Visualize augmented images
def visualize_augmented_images(dataloader):
    images, labels = next(iter(dataloader))
    fig, axes = plt.subplots(1, len(images[:5]), figsize=(15, 5))

    for i in range(5):
        img = images[i].permute(1, 2, 0).numpy()  # Convert from (C, H, W) to (H, W, C)
        img = img * 0.5 + 0.5  # Denormalize the image for visualization

        axes[i].imshow(img)
        axes[i].set_title(image_dataset.classes[labels[i]])
        axes[i].axis("off")

    plt.show()

visualize_augmented_images(dataloader)

# Apply more custom augmentations
print("Applying custom augmentation and transformations")
custom_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random color jitter
    transforms.RandomAffine(degrees=30, translate=(0.1, 0.1), scale=(0.9, 1.1)),  # Random affine transformations
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(20),  # Random rotation of up to 20 degrees
    transforms.ToTensor(),  # Convert image to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize the image
])

# Create a dataset with custom augmentations
augmented_dataset = datasets.ImageFolder(root=data_dir, transform=custom_transform)
augmented_loader = torch.utils.data.DataLoader(augmented_dataset, batch_size=16, shuffle=True)

# Visualize augmented images with custom transformations
visualize_augmented_images(augmented_loader)

# Feature engineering: Compute mean and standard deviation of images in the dataset
def compute_features(loader):
    features = []
    for images, labels in loader:
        for img in images:
            mean = img.mean().item()
            std = img.std().item()
            features.append((mean, std))
    return features

features = compute_features(dataloader)

# Print feature engineering results
print("Feature Engineering Results:")
for i, (mean, std) in enumerate(features[:5]):
    print(f"Image {i+1}: Mean = {mean:.4f}, Std = {std:.4f}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import tree
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from graphviz import Digraph

# an image dataset by converting image files into numerical values suitable for machine learning models.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd

# Specify the directory where the images are stored
image_dir = '/kaggle/input/mango-leafbd/MangoLeafBD Dataset'  # Replace with your directory path
categories = os.listdir(image_dir)  # List of subfolder names (i.e., class labels)

# Load the images and labels
data = []    # Stores numerical representations (flattened pixel values) of images.
labels = []   # Stores the class/category each image belongs to.

for category in categories:
    category_path = os.path.join(image_dir, category)
    for image_file in os.listdir(category_path):
        image_path = os.path.join(category_path, image_file)

        # Read the image
        image = cv2.imread(image_path)

        # Convert the image from BGR (OpenCV default) to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Resize image to 64x64 (you can change this size as needed)
        image = cv2.resize(image, (64, 64))

        # Flatten the image from 3D to 1D
        image = image.flatten()

        data.append(image)
        labels.append(category)

# Convert the list of data into a numpy array
X = np.array(data)  # Features (flattened images)
y = np.array(labels)  # Labels (class/category)

# Create a DataFrame with the image data and labels
df = pd.DataFrame(X)
df['label'] = y  # Add labels as a new column

# Check the shape of the data and print the first few rows
print(df.head())
print(f"Shape of feature matrix (X): {X.shape}")
print(f"Shape of labels (y): {y.shape}")

# Now `X` contains the numerical representations of the images and `y` contains their corresponding labels.


In [ ]:

print(df.shape)
print(df.describe())

# Train & Test

In [ ]:
from sklearn.model_selection import train_test_split

# Define the features (X) and the target (y)
X = df.drop(columns=['label'])  # Drop the 'label' column to get features
y = df['label']  # Target variable

# Split the dataset into 80% training and 20% testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Check the shape of train and test sets
print(f"Training set shape: {X_train.shape}, {y_train.shape}")
print(f"Testing set shape: {X_test.shape}, {y_test.shape}")


# Decision tree classifier usning

In [ ]:
# Create a decision tree classifier with gini criterion and maximum depth of 3
dt = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=42)
# Fit the model on the train data
dt.fit(X_train, y_train)
# Make predictions on the test data
y_pred = dt.predict(X_test)

#  Evaluate and compare performance of Decision tree

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt

# Make predictions with the Decision Tree model (or Random Forest if you are comparing both)
y_pred = dt.predict(X_test)  # This is assuming 'dt' is your Decision Tree model

# Calculate the accuracy, precision, recall, and f1-score
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, average='macro')
rec = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

# Print the results
print(f"Accuracy: {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall: {rec:.2f}")
print(f"F1-score: {f1:.2f}")

# Plot the confusion matrix
cm = confusion_matrix(y_test, y_pred)
plt.imshow(cm, cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.colorbar()
plt.show()

# Print the classification report
print(classification_report(y_test, y_pred))


# plot Decision tree

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# Assuming `df` is the DataFrame that contains the Mango Leaf dataset

# Prepare feature matrix (X) and target vector (y)
X = df.drop(columns=['label']).values  # Features (flattened image data)
y = df['label'].values  # Target labels

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the Decision Tree Classifier
dt = DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=42)
dt.fit(X_train, y_train)

# Assuming that the labels are numerical, convert them to categorical class names
# Replace 'label' with the corresponding column if necessary
class_names = list(df['label'].unique())  # This will get the unique class names from your 'label' column
feature_names = [f'Feature {i+1}' for i in range(X.shape[1])]  # Assuming features are unnamed, this will generate 'Feature 1', 'Feature 2', etc.

# Plot the decision tree
plt.figure(figsize=(20, 25))# 12, 8
plot_tree(dt, feature_names=feature_names, class_names=[str(x) for x in class_names], filled=True)
plt.title('Decision Tree for Mango Leaf Dataset')
plt.show()



#  Random Forest Creation and Training

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize Random Forest model
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

# Make predictions
y_pred_rf = rf_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred_rf))
